# ToDo

In [ ]:
#@title ToDo
#@markdown To start the application, click the play button to the right

# Source Code From: https://gist.github.com/emakryo/98142e8f5b02206ada660879f935139f


import ipywidgets as iw
import pickle

class ToDo:
    def __init__(self, load='todo', save=None):
        def _add(text):
            hb = self._add(text.value)
            self._state.append(hb)
            self._update()

        self._new_title = iw.Text(value='', placeholder='What needs to be done')
        self._new_title.on_submit(_add)
        self._visible = iw.Dropdown(options=['All', 'Active', 'Complete'])
        self._visible.observe(self._update, 'value')
        self._state = self._load(load)
        
        if type(save) is str:
            self._save = save
        else:
            self._save = load

        self._list = iw.VBox(self._state)      
        self._box = iw.VBox([self._new_title, self._visible, self._list])
        
    def _load(self, load):
        try:
            with open(load, 'rb') as f:
                state = pickle.load(f)
        except FileNotFoundError:
            return []
        
        return [self._add(s['name'], s['done']) for s in state]
        
    def _add(self, name, done=False):
        name = iw.Text(value=name, placeholder='ToDo')
        done = iw.ToggleButton(value=done, description='Done')
        done.observe(self._update, 'value')
        delete = iw.Button(description='Delete')
        delete.on_click(self._delete)
        hb = iw.HBox([name, done, delete])
        delete.hb = hb
        self._new_title.value = ''
        return hb
    
    def _delete(self, button):
        i = self._state.index(button.hb)
        del self._state[i]
        self._update()
        
    def _update(self, change=None):
        if self._visible.value == 'All':
            self._list.children = self._state
        elif self._visible.value == 'Active':
            self._list.children = [s for s in self._state if not s.children[1].value]
        else:
            self._list.children = [s for s in self._state if s.children[1].value]

        state = [{'name': s.children[0].value, 'done': s.children[1].value}
                 for s in self._state]
        with open(self._save, 'wb') as f:
            pickle.dump(state, f)
            
    def _ipython_display_(self):
        self._box._ipython_display_()


todo = ToDo()
todo